In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import os
import pdb

In [29]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

RuntimeError: Physical devices cannot be modified after being initialized

In [5]:
def get_model(maxlen, num_chars, num_layers, num_units):
    print('Build model...')
    model = Sequential()
    for layer_idx in range(num_layers):
        if layer_idx == 0:
            #the size is maxlen x numchars. Maxlen is because of the length of a sentence (128 words) while num_chars=119
            #which are the unique words in my vocabulary.
            #NB: the input is one hot encoded over the vocabulary size (num_chars)
            model.add(LSTM(num_units, return_sequences=True, input_shape=(maxlen, num_chars)))
        else:
            model.add(LSTM(num_units, return_sequences=False))
        model.add(Dropout(0.2))

    model.add(Dense(num_chars))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [6]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

In [24]:
def run(is_character=False, maxlen=None, num_units=None, model_prefix=''):

    character_mode = is_character

    if character_mode:
        if maxlen == None:
            maxlen = 1024
        if num_units == None:
            num_units = 32
        step = 2*17 # step to create training data for truncated-BPTT
    else: # word mode
        if maxlen == None:
            maxlen = 128 # maxlength used in RNN input
        if num_units == None: 
            num_units = 512 #number of unit per layer LSTM 512 
        step = 8

    if character_mode:
        num_char_pred = maxlen*3/2
    else: 
        num_char_pred = 17*30 #this should be the number of elements predicted in the output. How "long" is my output sequence

    num_layers = 2
    # 
    if character_mode:
        prefix = 'char'
    else:
        prefix = 'word'

    path = 'metallica_drums_text.txt' # Corpus file
    text = open(path).read()
    print('corpus length:', len(text))

    if character_mode:
        chars = set(text)
    else:
        chord_seq = text.split(' ')
        chars = set(chord_seq) #contains the unique words in my dictionary. They are 119
        text = chord_seq #contains the full text in an array format. Each entry of my array is a word of type 0xb0110101010 

    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    num_chars = len(char_indices) #number of unique words in my training set
    print('total chars:', num_chars)

    # cut the text in semi-redundant sequences of maxlen characters

    sentences = []
    next_chars = []
    #Here im creating the inputs and targets for my RNN. Each single input has length maxlen.
    #Inputs are semi-redundant, in the sense that i take a length of maxlen=128 and the step is 8. So my first part of the input
    #will be the same and the last 8 elements are "new". I'm just "slitting" of 8 notes ahead
    for i in range(0, len(text) - maxlen, step): #iterates over the range with steps of 8.
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])
    print('nb sequences:', len(sentences))
    print('Vectorization...')
    
    #Here i'm creating the input dataset and target dataset for my network
    #X is a tri-dimensional vector: 1 dimension -> Sentences, 2 dimension -> Single sentence, 3 dimension -> one hot encoded vector of the single word
    #So basically i have a structure where i have N sentences of maxlen Words where each word is represented as a one hot vector of length num_chars
    X = np.zeros((len(sentences), maxlen, num_chars), dtype=np.bool) #Input matrix
    y = np.zeros((len(sentences), num_chars), dtype=np.bool) #Target Matrix
    #Here i'm actually "populating" the matrixes, which were initialized with all zeros
    print('Entering initialization cycle')

    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1 #NB: char in this case means a whole word like oxb01011101. With char_indices[char] i'm retrieving the index of my word inside my dictionary of (words,index)
        print('Finished input initialization')
        y[i, char_indices[next_chars[i]]] = 1
    print('Completed Vectorization')
    # build the model: 2 stacked LSTM
    model = get_model(maxlen, num_chars, num_layers, num_units) 
    
    #Just some string declarations for folders management and names.
    #NB: CHANGE THE / with \ for windows! 
    result_directory = 'r_%s_%s_%d_%d_units' % (prefix, model_prefix, maxlen, num_units)
    filepath_model = os.path.join(result_directory, 'best_model.hdf')

    #filepath_model = '%sbest_model.hdf' % result_directory
    description_model = '%s, %d layers, %d units, %d maxlen, %d steps' % (prefix, num_layers, num_units, maxlen, step)
    
    #Usual Model checkpoints and Early Stopping
    checker = tf.keras.callbacks.ModelCheckpoint(filepath_model, monitor='loss', verbose=0, save_best_only=True, mode='auto')
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15, verbose=0, mode='auto')
    
    #create a result directory if it doesn't exist
    if not os.path.exists(result_directory):
        os.mkdir(result_directory)

    # write a description file.
    #creates an empty file with the drscription of my model as title
    with open(result_directory+description_model, 'w') as f_description:
        pass

    # train the model, output generated text after each iteration
    batch_size = 128 #Size of a training batch. So basically i'll update my loss function every 128 input sentences (usual batch gradient descent)
    loss_history = []
    pt_x = [5]
    #An epoch is a complete iteration over the whole input training set. So 10 epochs means that i iterates 10 times over my input dataset
    nb_epochs = [np.sum(pt_x[:i+1]) for i in range(len(pt_x))] #array containing many epochs length. The model will be fitted many times, one for each nb_epochs.

    # not random seed, but the same seed for all.
    #A random seed (or seed state, or just seed) is a number (or vector) used to initialize a pseudorandom number generator.
    start_index = random.randint(0, len(text) - maxlen - 1)

    for iteration, nb_epoch in zip(pt_x,nb_epochs):
        if os.path.exists('stop_asap.keunwoo'):
            os.remove('stop_asap.keunwoo')
            break

        print('-' * 50)
        print('Iteration', iteration)
        
        #fitting model over nb_epochs
        #result = model.fit(X, y, batch_size=batch_size, nb_epoch=nb_epoch, callbacks=[checker, early_stop]) 
        #loss_history = loss_history + result.history['loss']

        print ('Saving model after %d epochs...' % nb_epoch)
        #Saving model weights. Saving a model trained over nb_epochs
        #model.save_weights('%smodel_after_%d.hdf'%(result_directory, nb_epoch), overwrite=True) 
        model.load_weights("model_weights.h5")
        w2 = model.get_weights()
        print(w2)

        
        for diversity in [0.9, 1.0, 1.2]:
            #creates a .txt file where i will save my predictions
            with open(('%sresult_%s_iter_%02d_diversity_%4.2f.txt' % (result_directory, prefix, iteration, diversity)), 'w') as f_write:

                print()
                print('----- diversity:', diversity)
                f_write.write('diversity:%4.2f\n' % diversity)
                if character_mode:
                    generated = ''
                else:
                    generated = [] #simple initialization
                #selects a random sentence from my input dataset.
                sentence = text[start_index: start_index + maxlen]
                seed_sentence = text[start_index: start_index + maxlen]

                if character_mode:
                    generated += sentence
                else:
                    #at first iteration i just add my input sentence in my generated element
                    generated = generated + sentence


                print('----- Generating with seed:')

                if character_mode:
                    print(sentence)
                    sys.stdout.write(generated)
                else:
                    print(' '.join(sentence))

                for i in range(num_char_pred): 
                    # if generated.endswith('_END_'):
                    # 	break
                    x = np.zeros((1, maxlen, num_chars)) #initialization of input. Matrix of maxlen words, each 

                    for t, char in enumerate(sentence):
                        x[0, t, char_indices[char]] = 1. 

                    preds = model.predict(x, verbose=0)[0] 
                    next_index = sample(preds, diversity)
                    next_char = indices_char[next_index]

                    if character_mode:
                        generated += next_char
                        sentence = sentence[1:] + next_char
                    else:
                        generated.append(next_char)
                        sentence = sentence[1:]
                        sentence.append(next_char)

                    if character_mode:
                        sys.stdout.write(next_char)
                    # else:
                    # 	for ch in next_char:
                    # 		sys.stdout.write(ch)	

                    sys.stdout.flush()

                if character_mode:
                    f_write.write(seed_sentence + '\n')
                    f_write.write(generated)
                else:
                    f_write.write(' '.join(seed_sentence) + '\n')
                    f_write.write(' ' .join(generated))

        np.save('%sloss_%s.npy'%(result_directory, prefix), loss_history)

    print ('Done! You might want to run main_post_process.py to get midi files. ')
    print ('You need python-midi (https://github.com/vishnubob/python-midi) to run it.')


In [25]:
run()

corpus length: 2141692
total chars: 119
nb sequences: 23204
Vectorization...
Entering initialization cycle
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initiali

Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished i

Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished i

Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished i

Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished i

...

Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished i

Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished i

Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished input initialization
Finished i

--------------------------------------------------
Iteration 5
Saving model after 5 epochs...
[array([[-0.10083451, -0.03838838, -0.12586473, ...,  0.18239225,
        -0.08547749, -0.01342126],
       [ 0.04214023,  0.00835916, -0.03349414, ...,  0.01632392,
        -0.03146268, -0.01877232],
       [ 0.04110439,  0.0213888 ,  0.01764079, ...,  0.05062227,
         0.04571038, -0.02526665],
       ...,
       [ 0.07436422, -0.03707097, -0.00176886, ...,  0.03771044,
        -0.04669984, -0.06712974],
       [ 0.03118392, -0.05031696,  0.11468485, ..., -0.06798497,
        -0.01634006,  0.26103008],
       [ 0.00271396,  0.03249216, -0.06014514, ...,  0.14778799,
        -0.05118079, -0.15737256]], dtype=float32), array([[ 0.00720661, -0.02321257, -0.0002803 , ..., -0.04135935,
        -0.00527885,  0.00944946],
       [-0.04792819,  0.01825438,  0.01644409, ...,  0.03466489,
         0.00986013, -0.01870644],
       [ 0.00418356,  0.01639651, -0.0450618 , ...,  0.02120958,
        -0.

UnknownError:  [_Derived_]  Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_7/lstm_14/StatefulPartitionedCall]] [Op:__inference_distributed_function_12667]

Function call stack:
distributed_function -> distributed_function -> distributed_function


In [23]:

model.load_weights("model_weights.h5")

NameError: name 'model' is not defined